# Starling Task across subject behavioral analysis combined
Let's start the analysis based on Rhiannon's list [here](https://uofutah-my.sharepoint.com/:w:/g/personal/u1363968_umail_utah_edu/ESn4E7plikFIs1ZyLHy5YaUBZfn_td7fv2yCh6I5HsWL2g?e=MxDJfG&CID=0428038a-a81f-6b6f-5c00-c8f4ada097eb).

By: Niloufar Shahdoust

In [26]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
matplotlib.use('svg')
matplotlib.rcParams['svg.fonttype'] = 'none'
matplotlib.rcParams['font.weight'] = 'bold'
from matplotlib.patches import Patch
import os
import seaborn as sns 
from matplotlib.ticker import MaxNLocator
from matplotlib.ticker import FixedLocator
from scipy.stats import permutation_test
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from scipy.stats import pearsonr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Patch
from scipy.optimize import curve_fit
from statsmodels.stats.multitest import multipletests
import os
import matplotlib.ticker as mticker
from scipy.stats import mannwhitneyu


# ****************************************************************************
## reading all subjects data

In [2]:
folder_path = 'data_risk_added'
folder_path_epileptic = 'data_risk_added_epileptic'

output_folder = '30_RL_agent_TD_learn_healthy_vs_epileptic'

if not os.path.exists(output_folder):
    os.makedirs(output_folder)

if not os.path.exists(folder_path_epileptic):
    os.makedirs(folder_path_epileptic)

dataframes = []
dataframes_epileptic = []

for file_name in os.listdir(folder_path):
    if file_name.endswith('.xlsx'):
        file_path = os.path.join(folder_path, file_name)
        df = pd.read_excel(file_path)
        dataframes.append(df)



for file_name in os.listdir(folder_path_epileptic):
    if file_name.endswith('.csv'):
        file_path = os.path.join(folder_path_epileptic, file_name)
        df = pd.read_csv(file_path)
        dataframes_epileptic.append(df)
        

# ****************************************************************************

In [3]:
for df in dataframes:
    df['block_type'] = None
    df.loc[df['block'] == 4, 'block_type'] = 'mix'              # block 4 is mix
    df.loc[df['block'].isin([1, 2, 3]), 'block_type'] = 'fix'   # else is fix

    

for df in dataframes_epileptic:
    df['block_type'] = None

    df.loc[df['block'] == 4, 'block_type'] = 'mix'              # block 4 is mix
    df.loc[df['block'].isin([1, 2, 3]), 'block_type'] = 'fix'   # else is fix

In [4]:
for df in dataframes:
    df.drop(df[df['arrowRT'] == 'na'].index, inplace=True)
    df.reset_index(drop=True, inplace=True)

for df in dataframes_epileptic:
    df.drop(df[df['arrowRT'] == 'na'].index, inplace=True)
    df.reset_index(drop=True, inplace=True)


In [5]:
dataframes_epileptic[0]

,arrowRT,distribution,interTrialInterval,outcome,myCard,yourCard,spaceRT,totalReward,trialIndex,trialType,choice,block,timeoutRepeat,is_within_IQR,risk,block_type
0,2390,uniform,756,win,2,7,3186,10.5,0,response,arrowdown,1,0,0,0.125,fix
1,1262,uniform,874,win,9,4,1040,11,1,response,arrowup,1,0,0,0.000,fix
2,1547,uniform,791,win,5,6,204,11.5,2,response,arrowdown,1,0,1,0.500,fix
3,1627,uniform,828,win,9,1,872,12,3,response,arrowup,1,0,0,0.000,fix
4,459,uniform,894,win,6,4,790,12.5,4,response,arrowup,1,0,0,0.375,fix
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
265,514,high,847,win,8,7,429,85,134,response,arrowup,4,0,1,0.243,mix
266,179,high,902,win,4,8,211,85.5,115,response,arrowdown,4,0,1,0.146,mix
267,419,low,815,lose,8,9,353,85,81,response,arrowup,4,0,1,0.023,mix
268,731,low,941,lose,3,1,212,84.5,59,response,arrowdown,4,0,1,0.447,mix


## number of participants

In [6]:
n_participant = len(dataframes)
print(f"there are {n_participant} healthy participants.")


n_participant_epileptic = len(dataframes_epileptic)
print(f"there are {n_participant_epileptic}  epileptic participants.")

there are 38 healthy participants.
there are 10  epileptic participants.


### visualization prerequisites:
this order is very important in adding all the analysis block labels!

In [7]:
x_labels = ['uniform','low', 'high']
distributions_to_show = ['uniform','low', 'high']
colors = ['#808080',  '#ff7f0e', '#2ca02c']

# total reward

In [52]:
fig, axes = plt.subplots(1,2, figsize=(3.5,3), sharey=True)

# --- Healthy Participants ---
participant_totalReward_list = []

for df in dataframes:
    participant_totalReward = df[df['totalReward'] != "na"]['totalReward'].astype(float).tolist()
    participant_totalReward_list.append(participant_totalReward)
    axes[0].plot(participant_totalReward, color='black', linewidth = 0.25, alpha=0.3)

mean_total_reward = np.mean(participant_totalReward_list, axis=0)
axes[0].plot(mean_total_reward, color='black',  linewidth=0.5, alpha=1)
axes[0].set_title("healthy")
axes[0].set_xlabel("trial")
axes[0].set_ylabel("total reward ($)")
axes[0].spines['top'].set_visible(False)
axes[0].spines['right'].set_visible(False)

# --- Epileptic Participants ---
participant_totalReward_list_epileptic = []

for df in dataframes_epileptic:
    participant_totalReward = df[df['totalReward'] != "na"]['totalReward'].astype(float).tolist()
    participant_totalReward_list_epileptic.append(participant_totalReward)
    axes[1].plot(participant_totalReward, color='black', linewidth = 0.25, alpha=0.3)

mean_total_reward_epileptic = np.mean(participant_totalReward_list_epileptic, axis=0)
axes[1].plot(mean_total_reward_epileptic, color='black', linewidth=0.5, alpha=1)
axes[1].set_title("epileptic")
axes[1].set_xlabel("trial")
axes[1].spines['top'].set_visible(False)
axes[1].spines['right'].set_visible(False)

# --- Permutation Test ---
healthy_means = np.mean(participant_totalReward_list, axis=1)
epileptic_means = np.mean(participant_totalReward_list_epileptic, axis=1)
observed_diff = np.mean(healthy_means) - np.mean(epileptic_means)

combined = np.concatenate([healthy_means, epileptic_means])
n_healthy = len(healthy_means)
n_permutations = 10000
permuted_diffs = []

np.random.seed(0)
for _ in range(n_permutations):
    permuted = np.random.permutation(combined)
    group1 = permuted[:n_healthy]  # healthy group
    group2 = permuted[n_healthy:]  # epileptic group
    diff = np.mean(group1) - np.mean(group2)
    permuted_diffs.append(diff)

permuted_diffs = np.array(permuted_diffs)
p_value = np.mean(np.abs(permuted_diffs) >= np.abs(observed_diff))

# --- Titles & Save ---
main_title = "total reward across trials"
result_text = f"p = {p_value:.2f}; n.s."
plt.suptitle(f"{main_title}\n{result_text}", fontsize=16)

for ax in axes:
    ax.set_ylim(0, 100)
    ax.set_yticks(np.arange(0, 101, 20))  


plt.tight_layout(rect=[0, 0.08, 1, 0.95])
plt.savefig(os.path.join(output_folder, "total_reward.pdf"), format="pdf", dpi=500, bbox_inches="tight")
plt.savefig(os.path.join(output_folder, "total_reward.svg"), format="svg", dpi=500, bbox_inches="tight")
plt.show()


C:\Users\Nill\AppData\Local\Temp\1\ipykernel_34212\2429027755.py:68: UserWarning: FigureCanvasSVG is non-interactive, and thus cannot be shown
  plt.show()


# reaction times across trials

In [ ]:
def compute_trialwise_means(df_list, rt_col, max_rt=3000, min_rt=1e-9):
    trial_dfs = []
    for pid, df in enumerate(df_list):
        d = df.copy().reset_index(drop=True)

        if rt_col not in d.columns:
            continue

        d[rt_col] = pd.to_numeric(d[rt_col], errors='coerce')
        d = d.dropna(subset=[rt_col])
        d = d[(d[rt_col] > min_rt) & (d[rt_col] <= max_rt)]

        if d.empty:
            continue

        # assign trial index
        d['trial'] = np.arange(1, len(d) + 1)

        g = d[['trial', rt_col]].copy()
        g['participant'] = pid
        trial_dfs.append(g)

    if not trial_dfs:
        return pd.DataFrame(columns=['trial', f'{rt_col}_mean', f'{rt_col}_sem'])

    all_data = pd.concat(trial_dfs, ignore_index=True)
    summary = all_data.groupby('trial')[rt_col].agg(['mean', 'sem']).reset_index()
    summary.rename(columns={'mean': f'{rt_col}_mean', 'sem': f'{rt_col}_sem'}, inplace=True)
    return summary

# -------------------------------------------------

def smooth_series(y, window=10):
    if isinstance(y, pd.Series):
        values = y.values
    else:
        values = np.asarray(y)
    kernel = np.ones(window) / window
    smoothed = np.convolve(values, kernel, mode='same')
    return pd.Series(smoothed, index=(y.index if isinstance(y, pd.Series) else None))

# -------------------------------------------------
def prepare_summary(df_list, rt_col):
    summary = compute_trialwise_means(df_list, rt_col)
    if summary.empty:
        return summary
    summary[f'{rt_col}_mean_smooth'] = smooth_series(summary[f'{rt_col}_mean'], window=5)
    summary[f'{rt_col}_sem_smooth']  = smooth_series(summary[f'{rt_col}_sem'], window=5)
    return summary

# -------------------------------------------------
# Helper to add vertical lines and labels
def add_block_lines(ax, trials, labels):
    ymax = ax.get_ylim()[1]
    for t, label in zip(trials, labels):
        ax.axvline(x=t, color='gray', linestyle='--', linewidth=0.5)
        ax.text(t, ymax, label, ha='center', va='bottom',
                fontsize=8, fontweight='bold')

# -------------------------------------------------
# Healthy and epileptic separately
space_summary_healthy = prepare_summary(dataframes, 'spaceRT')
arrow_summary_healthy = prepare_summary(dataframes, 'arrowRT')

space_summary_epileptic = prepare_summary(dataframes_epileptic, 'spaceRT')
arrow_summary_epileptic = prepare_summary(dataframes_epileptic, 'arrowRT')

# Trials and labels
block_trials = [1, 46, 91, 136]
# block_labels = ['b1', 'b2', 'b3', 'b4']
block_labels = [' ', ' ', ' ', ' ']
# -------------------------------------------------
# Plot 2×2
fig, axes = plt.subplots(2, 2, figsize=(4,4.5), dpi=300, sharey=True)

# --- Healthy SpaceRT
ax = axes[0, 0]
ax.plot(space_summary_healthy['trial'], space_summary_healthy['spaceRT_mean_smooth'],
        color='black', label='spaceRT', linewidth=0.5)
ax.fill_between(space_summary_healthy['trial'],
                space_summary_healthy['spaceRT_mean_smooth'] - space_summary_healthy['spaceRT_sem_smooth'],
                space_summary_healthy['spaceRT_mean_smooth'] + space_summary_healthy['spaceRT_sem_smooth'],
                color='black', alpha=0.2)
ax.set_title("flip RT", fontsize=8)
ax.set_ylabel("RT (ms) - healthy",  fontsize=8)
ax.set_xlabel("trial", fontsize=8)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.yaxis.set_major_locator(MaxNLocator(nbins=6, integer=False))
add_block_lines(ax, block_trials, block_labels)

# --- Healthy ArrowRT
ax = axes[0, 1]
ax.plot(arrow_summary_healthy['trial'], arrow_summary_healthy['arrowRT_mean_smooth'],
        color='black', label='arrowRT', linewidth=0.5)
ax.fill_between(arrow_summary_healthy['trial'],
                arrow_summary_healthy['arrowRT_mean_smooth'] - arrow_summary_healthy['arrowRT_sem_smooth'],
                arrow_summary_healthy['arrowRT_mean_smooth'] + arrow_summary_healthy['arrowRT_sem_smooth'],
                color='black', alpha=0.2)
ax.set_title("choice RT", fontsize=8)
ax.set_xlabel("trial", fontsize=8)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.yaxis.set_major_locator(MaxNLocator(nbins=6, integer=False))
add_block_lines(ax, block_trials, block_labels)

# --- Epileptic SpaceRT
ax = axes[1, 0]
ax.plot(space_summary_epileptic['trial'], space_summary_epileptic['spaceRT_mean_smooth'],
        color='black', label='spaceRT', linewidth=0.5)
ax.fill_between(space_summary_epileptic['trial'],
                space_summary_epileptic['spaceRT_mean_smooth'] - space_summary_epileptic['spaceRT_sem_smooth'],
                space_summary_epileptic['spaceRT_mean_smooth'] + space_summary_epileptic['spaceRT_sem_smooth'],
                color='black', alpha=0.2)
ax.set_title("flip RT", fontsize=8)
ax.set_ylabel("RT (ms) - epileptic", fontsize=8)
ax.set_xlabel("trial", fontsize=8)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.yaxis.set_major_locator(MaxNLocator(nbins=6, integer=False))
add_block_lines(ax, block_trials, block_labels)

# --- Epileptic ArrowRT
ax = axes[1, 1]
ax.plot(arrow_summary_epileptic['trial'], arrow_summary_epileptic['arrowRT_mean_smooth'],
        color='black', label='arrowRT', linewidth=0.5)
ax.fill_between(arrow_summary_epileptic['trial'],
                arrow_summary_epileptic['arrowRT_mean_smooth'] - arrow_summary_epileptic['arrowRT_sem_smooth'],
                arrow_summary_epileptic['arrowRT_mean_smooth'] + arrow_summary_epileptic['arrowRT_sem_smooth'],
                color='black', alpha=0.2)
ax.set_title("choice RT", fontsize=8)
ax.set_xlabel("trial", fontsize=8)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.yaxis.set_major_locator(MaxNLocator(nbins=6, integer=False))
add_block_lines(ax, block_trials, block_labels)



main_title = "RTs across trials"
plt.suptitle(f"{main_title}", fontsize=12)



# -------------------------------------------------
plt.tight_layout()
os.makedirs(output_folder, exist_ok=True)
plt.savefig(os.path.join(output_folder, "trialwise_space_arrow_RT.pdf"),
            format="pdf", dpi=1200, bbox_inches="tight")
plt.savefig(os.path.join(output_folder, "trialwise_space_arrow_RT.svg"),
            format="svg", dpi=1200, bbox_inches="tight")
plt.show()


C:\Users\Nill\AppData\Local\Temp\1\ipykernel_34212\150598223.py:157: UserWarning: FigureCanvasSVG is non-interactive, and thus cannot be shown
  plt.show()


# accuracy

In [10]:

def add_sig_bar(ax, pos1, pos2, y, pval, bar_height=0.02, text_offset=0.02):
    alpha = 0.05
    ax.plot([pos1, pos1, pos2, pos2],
            [y, y+bar_height, y+bar_height, y],
            lw=0.7, c='black')

    if pval < (alpha/3): # there are 3 groups
        text = "*"
    else:
        text = "n.s."

    ax.text((pos1 + pos2) / 2, y + bar_height + text_offset,
            text, ha='center', va='bottom', fontsize=8)

# -------------------------------------------------
fig, ax = plt.subplots(figsize=(4,4), dpi=300)

# --- Collect values ---
healthy_vals_all = {dist: [] for dist in distributions_to_show}
epileptic_vals_all = {dist: [] for dist in distributions_to_show}

for df in dataframes:
    df['is_win'] = df['outcome'].apply(lambda x: 1 if x == 'win' else 0)
    for dist in distributions_to_show:
        healthy_vals_all[dist].append(df[df['distribution'] == dist]['is_win'].mean())

for df in dataframes_epileptic:
    df['is_win'] = df['outcome'].apply(lambda x: 1 if x == 'win' else 0)
    for dist in distributions_to_show:
        epileptic_vals_all[dist].append(df[df['distribution'] == dist]['is_win'].mean())

# --- Plotting ---
positions = []
labels = []
p_values = []
bar_width = 0.3
spacing = 2.0           # gap between distributions
group_gap = 0.3         # extra gap between healthy & epileptic within same distribution

for i, dist in enumerate(distributions_to_show):
    # shift positions left/right by group_gap
    pos_healthy = i * spacing - (bar_width/2 + group_gap/2)
    pos_epileptic = i * spacing + (bar_width/2 + group_gap/2)

    positions.extend([pos_healthy, pos_epileptic])
    labels.extend([f"healthy", f"epileptic"])

    # --- Healthy box ---
    ax.boxplot(
        healthy_vals_all[dist],
        positions=[pos_healthy],
        widths=bar_width,
        patch_artist=True,
        boxprops=dict(facecolor='none', color='black', linewidth=0.5),
        medianprops=dict(color='black', linewidth=0.5),
        whiskerprops=dict(color='black', linewidth=0.5),
        capprops=dict(color='black', linewidth=0.5),
        showfliers=False
    )
    jitter_x = np.random.normal(pos_healthy, 0.05, size=len(healthy_vals_all[dist]))
    jitter_y = np.array(healthy_vals_all[dist]) + np.random.normal(0, 0.005, size=len(healthy_vals_all[dist]))
    ax.scatter(jitter_x, jitter_y, s=8, color=colors[i], alpha=0.5, edgecolors='none')

    # --- Epileptic box ---
    ax.boxplot(
        epileptic_vals_all[dist],
        positions=[pos_epileptic],
        widths=bar_width,
        patch_artist=True,
        boxprops=dict(facecolor='none', color='black', linewidth=0.5),
        medianprops=dict(color='black', linewidth=0.5),
        whiskerprops=dict(color='black', linewidth=0.5),
        capprops=dict(color='black', linewidth=0.5),
        showfliers=False
    )
    jitter_x = np.random.normal(pos_epileptic, 0.05, size=len(epileptic_vals_all[dist]))
    jitter_y = np.array(epileptic_vals_all[dist]) + np.random.normal(0, 0.005, size=len(epileptic_vals_all[dist]))
    ax.scatter(jitter_x, jitter_y, s=8, color=colors[i], alpha=0.5, edgecolors='none')

    # --- Permutation Test ---
    healthy = healthy_vals_all[dist]
    epileptic = epileptic_vals_all[dist]
    combined = np.concatenate([healthy, epileptic])
    n_healthy = len(healthy)
    observed_diff = np.mean(healthy) - np.mean(epileptic)
    perm_diffs = [
        np.mean(np.random.permutation(combined)[:n_healthy]) -
        np.mean(np.random.permutation(combined)[n_healthy:])
        for _ in range(10000)
    ]
    p_val = np.mean(np.abs(perm_diffs) >= np.abs(observed_diff))
    p_values.append(p_val)

# --- Multiple Hypothesis Correction (Bonferroni) ---
reject, pvals_corrected, _, _ = multipletests(p_values, method='bonferroni')

# --- Add significance bars ---
y_sig = 1.02  # fixed y-height (2% above top=1)
for i, dist in enumerate(distributions_to_show):
    pos_healthy = i * spacing - (bar_width/2 + group_gap/2)
    pos_epileptic = i * spacing + (bar_width/2 + group_gap/2)
    add_sig_bar(ax, pos_healthy, pos_epileptic, y_sig, pvals_corrected[i])

# --- Final touches ---
ax.set_xticks(positions)
ax.set_ylim(0, 1.1)  # extend to make space for bars
ax.set_xticklabels(labels, rotation=45, ha='right')
ax.set_ylabel("accuracy (%)")
ax.set_title("accuracy")
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

# convert y-axis from 0–1 to 0–100
ax.yaxis.set_major_formatter(mticker.FuncFormatter(lambda y, _: f'{int(y*100)}'))

plt.tight_layout()
plt.savefig(os.path.join(output_folder, "accuracy.pdf"),
            format="pdf", dpi=300, bbox_inches="tight")
plt.savefig(os.path.join(output_folder, "accuracy.svg"),
            format="svg", dpi=300, bbox_inches="tight")
plt.show()


C:\Users\Nill\AppData\Local\Temp\1\ipykernel_34212\2454202720.py:121: UserWarning: FigureCanvasSVG is non-interactive, and thus cannot be shown
  plt.show()


# space RT

In [11]:
def add_sig_bar(ax, pos1, pos2, y, pval, bar_height=30, text_offset=0.02, lift=100):

    y = y + lift  # shift everything upward

    # bar
    ax.plot([pos1, pos1, pos2, pos2],
            [y, y+bar_height, y+bar_height, y],
            lw=0.5, c='black')

    # text
    if pval < 0.05:
        text = "*"
    else:
        text = "n.s."

    ax.text((pos1 + pos2) / 2, y + bar_height + text_offset,
            text, ha='center', va='bottom', fontsize=8)

def remove_outliers(series, thresh=3, upper_limit=4000):

    s = series.dropna()
    if s.empty:
        return s
    mean, std = s.mean(), s.std()
    filtered = s[(s >= mean - thresh*std) & (s <= mean + thresh*std)]
    filtered = filtered[filtered <= upper_limit]
    return filtered


# -------------------------------------------------
fig, ax = plt.subplots(figsize=(4,4), dpi=300)

# --- Collect values ---
healthy_vals_all = {dist: [] for dist in distributions_to_show}
epileptic_vals_all = {dist: [] for dist in distributions_to_show}

for df in dataframes:
    df['flipRT'] = pd.to_numeric(df['spaceRT'], errors='coerce')
    for dist in distributions_to_show:
        vals = remove_outliers(df[df['distribution'] == dist]['flipRT'])
        if not vals.empty:
            healthy_vals_all[dist].append(vals.mean())

for df in dataframes_epileptic:
    df['flipRT'] = pd.to_numeric(df['spaceRT'], errors='coerce')
    for dist in distributions_to_show:
        vals = remove_outliers(df[df['distribution'] == dist]['flipRT'])
        if not vals.empty:
            epileptic_vals_all[dist].append(vals.mean())

# --- Plotting ---
positions = []
labels = []
p_values = []
bar_width = 0.3
spacing = 2.0           # gap between distributions
group_gap = 0.3         # extra gap between healthy & epileptic within same distribution

for i, dist in enumerate(distributions_to_show):
    # shift positions left/right by group_gap
    pos_healthy = i * spacing - (bar_width/2 + group_gap/2)
    pos_epileptic = i * spacing + (bar_width/2 + group_gap/2)

    positions.extend([pos_healthy, pos_epileptic])
    labels.extend([f"healthy", f"epileptic"])

    # --- Healthy box ---
    ax.boxplot(
        healthy_vals_all[dist],
        positions=[pos_healthy],
        widths=bar_width,
        patch_artist=True,
        boxprops=dict(facecolor='none', color='black', linewidth=0.5),
        medianprops=dict(color='black', linewidth=0.5),
        whiskerprops=dict(color='black', linewidth=0.5),
        capprops=dict(color='black', linewidth=0.5),
        showfliers=False
    )
    jitter_x = np.random.normal(pos_healthy, 0.05, size=len(healthy_vals_all[dist]))
    jitter_y = np.array(healthy_vals_all[dist]) + np.random.normal(0, 1, size=len(healthy_vals_all[dist]))
    ax.scatter(jitter_x, jitter_y, s=8, color=colors[i], alpha=0.5, edgecolors='none')

    # --- Epileptic box ---
    ax.boxplot(
        epileptic_vals_all[dist],
        positions=[pos_epileptic],
        widths=bar_width,
        patch_artist=True,
        boxprops=dict(facecolor='none', color='black', linewidth=0.5),
        medianprops=dict(color='black', linewidth=0.5),
        whiskerprops=dict(color='black', linewidth=0.5),
        capprops=dict(color='black', linewidth=0.5),
        showfliers=False
    )
    jitter_x = np.random.normal(pos_epileptic, 0.05, size=len(epileptic_vals_all[dist]))
    jitter_y = np.array(epileptic_vals_all[dist]) + np.random.normal(0, 1, size=len(epileptic_vals_all[dist]))
    ax.scatter(jitter_x, jitter_y, s=8, color=colors[i], alpha=0.5, edgecolors='none')

    # --- Permutation Test ---
    healthy = healthy_vals_all[dist]
    epileptic = epileptic_vals_all[dist]
    combined = np.concatenate([healthy, epileptic])
    n_healthy = len(healthy)
    observed_diff = np.mean(healthy) - np.mean(epileptic)
    perm_diffs = [
        np.mean(np.random.permutation(combined)[:n_healthy]) -
        np.mean(np.random.permutation(combined)[n_healthy:])
        for _ in range(10000)
    ]
    p_val = np.mean(np.abs(perm_diffs) >= np.abs(observed_diff))
    p_values.append(p_val)

# --- Multiple Hypothesis Correction (Bonferroni) ---
reject, pvals_corrected, _, _ = multipletests(p_values, method='bonferroni')

# --- Add significance bars ---
y_sig = max(ax.get_ylim()) * 0.95
for i, dist in enumerate(distributions_to_show):
    pos_healthy = i * spacing - (bar_width/2 + group_gap/2)
    pos_epileptic = i * spacing + (bar_width/2 + group_gap/2)
    add_sig_bar(ax, pos_healthy, pos_epileptic, y_sig, pvals_corrected[i])

# --- Final touches ---
ax.set_xticks(positions)
ax.set_ylim(0, ax.get_ylim()[1]*1.1)  # extend to make space for bars
ax.set_xticklabels(labels, rotation=45, ha='right')
ax.set_ylabel("flip RT (ms)")
ax.set_title("flip RT")
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

plt.tight_layout()
plt.savefig(os.path.join(output_folder, "flipRT.pdf"),
            format="pdf", dpi=300, bbox_inches="tight")
plt.savefig(os.path.join(output_folder, "flipRT.svg"),
            format="svg", dpi=300, bbox_inches="tight")
plt.show()


C:\Users\Nill\AppData\Local\Temp\1\ipykernel_34212\413902562.py:137: UserWarning: FigureCanvasSVG is non-interactive, and thus cannot be shown
  plt.show()


# arrow RT

In [45]:
def add_sig_bar(ax, pos1, pos2, y, pval, bar_height=30, text_offset=0.02, lift=100):

    y = y + lift  # shift everything upward

    # bar
    ax.plot([pos1, pos1, pos2, pos2],
            [y, y+bar_height, y+bar_height, y],
            lw=0.5, c='black')

    # text
    if pval < 0.05:
        text = "*"
    else:
        text = "n.s."

    ax.text((pos1 + pos2) / 2, y + bar_height + text_offset,
            text, ha='center', va='bottom', fontsize=8)

    

def remove_outliers(series, thresh=3, upper_limit=4000):

    s = series.dropna()
    if s.empty:
        return s
    mean, std = s.mean(), s.std()
    filtered = s[(s >= mean - thresh*std) & (s <= mean + thresh*std)]
    filtered = filtered[filtered <= upper_limit]
    return filtered


# -------------------------------------------------
fig, ax = plt.subplots(figsize=(4,4), dpi=300)

# --- Collect values ---
healthy_vals_all = {dist: [] for dist in distributions_to_show}
epileptic_vals_all = {dist: [] for dist in distributions_to_show}

for df in dataframes:
    df['choiceRT'] = pd.to_numeric(df['arrowRT'], errors='coerce')
    for dist in distributions_to_show:
        vals = remove_outliers(df[df['distribution'] == dist]['choiceRT'])
        if not vals.empty:
            healthy_vals_all[dist].append(vals.mean())

for df in dataframes_epileptic:
    df['choiceRT'] = pd.to_numeric(df['arrowRT'], errors='coerce')
    for dist in distributions_to_show:
        vals = remove_outliers(df[df['distribution'] == dist]['choiceRT'])
        if not vals.empty:
            epileptic_vals_all[dist].append(vals.mean())

# --- Plotting ---
positions = []
labels = []
p_values = []
bar_width = 0.3
spacing = 2.0           # gap between distributions
group_gap = 0.3         # extra gap between healthy & epileptic within same distribution

for i, dist in enumerate(distributions_to_show):
    # shift positions left/right by group_gap
    pos_healthy = i * spacing - (bar_width/2 + group_gap/2)
    pos_epileptic = i * spacing + (bar_width/2 + group_gap/2)

    positions.extend([pos_healthy, pos_epileptic])
    labels.extend([f"healthy", f"epileptic"])

    # --- Healthy box ---
    ax.boxplot(
        healthy_vals_all[dist],
        positions=[pos_healthy],
        widths=bar_width,
        patch_artist=True,
        boxprops=dict(facecolor='none', color='black', linewidth=0.5),
        medianprops=dict(color='black', linewidth=0.5),
        whiskerprops=dict(color='black', linewidth=0.5),
        capprops=dict(color='black', linewidth=0.5),
        showfliers=False
    )
    jitter_x = np.random.normal(pos_healthy, 0.05, size=len(healthy_vals_all[dist]))
    jitter_y = np.array(healthy_vals_all[dist]) + np.random.normal(0, 1, size=len(healthy_vals_all[dist]))
    ax.scatter(jitter_x, jitter_y, s=8, color=colors[i], alpha=0.5, edgecolors='none')

    # --- Epileptic box ---
    ax.boxplot(
        epileptic_vals_all[dist],
        positions=[pos_epileptic],
        widths=bar_width,
        patch_artist=True,
        boxprops=dict(facecolor='none', color='black', linewidth=0.5),
        medianprops=dict(color='black', linewidth=0.5),
        whiskerprops=dict(color='black', linewidth=0.5),
        capprops=dict(color='black', linewidth=0.5),
        showfliers=False
    )
    jitter_x = np.random.normal(pos_epileptic, 0.05, size=len(epileptic_vals_all[dist]))
    jitter_y = np.array(epileptic_vals_all[dist]) + np.random.normal(0, 1, size=len(epileptic_vals_all[dist]))
    ax.scatter(jitter_x, jitter_y, s=8, color=colors[i], alpha=0.5, edgecolors='none')

    # --- Permutation Test ---
    healthy = healthy_vals_all[dist]
    epileptic = epileptic_vals_all[dist]
    combined = np.concatenate([healthy, epileptic])
    n_healthy = len(healthy)
    observed_diff = np.mean(healthy) - np.mean(epileptic)
    perm_diffs = [
        np.mean(np.random.permutation(combined)[:n_healthy]) -
        np.mean(np.random.permutation(combined)[n_healthy:])
        for _ in range(10000)
    ]
    p_val = np.mean(np.abs(perm_diffs) >= np.abs(observed_diff))
    p_values.append(p_val)

# --- Multiple Hypothesis Correction (Bonferroni) ---
reject, pvals_corrected, _, _ = multipletests(p_values, method='bonferroni')

# --- Add significance bars ---
y_sig = max(ax.get_ylim()) * 0.95
for i, dist in enumerate(distributions_to_show):
    pos_healthy = i * spacing - (bar_width/2 + group_gap/2)
    pos_epileptic = i * spacing + (bar_width/2 + group_gap/2)
    add_sig_bar(ax, pos_healthy, pos_epileptic, y_sig, pvals_corrected[i])

# --- Final touches ---
ax.set_xticks(positions)
ax.set_ylim(0, ax.get_ylim()[1]*1.1)  # extend to make space for bars
ax.set_xticklabels(labels, rotation=45, ha='right')
ax.set_ylabel("choice RT (ms)")
ax.set_title("choice RT")
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

plt.tight_layout()
plt.savefig(os.path.join(output_folder, "choiceRT.pdf"),
            format="pdf", dpi=300, bbox_inches="tight")
plt.savefig(os.path.join(output_folder, "choiceRT.svg"),
            format="svg", dpi=300, bbox_inches="tight")
plt.show()


C:\Users\Nill\AppData\Local\Temp\1\ipykernel_34212\4188688772.py:139: UserWarning: FigureCanvasSVG is non-interactive, and thus cannot be shown
  plt.show()


# flipRT and choiceRT for same vs. different trials in mix block

SAME TRIALS:

taking the rows that their previous row's card is from the same distribution

DIFFERENT TRIALS:

taking the rows that their previous row's card is from a different distribution



In [36]:
# taking mix block type cause we need this
dataframes_epileptic_mix = [
    df[df['block_type'] == 'mix'].copy()
    for df in dataframes_epileptic]


dataframes_mix = [
    df[df['block_type'] == 'mix'].copy()
    for df in dataframes]


def split_same_diff(df):
    df = df.copy().reset_index(drop=True)
    if 'distribution' not in df.columns:
        return pd.DataFrame(), pd.DataFrame()

    # Shift distribution column to compare with previous trial
    df['prev_dist'] = df['distribution'].shift(1)

    same_df = df[df['distribution'] == df['prev_dist']].copy()
    diff_df = df[df['distribution'] != df['prev_dist']].copy()

    return same_df, diff_df


dataframes_epileptic_same = []
dataframes_epileptic_diff = []

for df in dataframes_epileptic:
    mix_df = df[df['block_type'] == 'mix'].copy()
    same_df, diff_df = split_same_diff(mix_df)
    dataframes_epileptic_same.append(same_df)
    dataframes_epileptic_diff.append(diff_df)



dataframes_same = []
dataframes_diff = []


for df in dataframes:
    mix_df = df[df['block_type'] == 'mix'].copy()
    same_df, diff_df = split_same_diff(mix_df)
    dataframes_same.append(same_df)
    dataframes_diff.append(diff_df)


def collect_means(df_list, rt_col):
    vals_all = {dist: [] for dist in distributions_to_show}
    for df in df_list:
        df[rt_col] = pd.to_numeric(df[rt_col], errors='coerce')
        for dist in distributions_to_show:
            vals = remove_outliers(df[df['distribution'] == dist][rt_col])
            if not vals.empty:
                vals_all[dist].append(vals.mean())
    return vals_all

healthy_space = collect_means(dataframes_same, 'spaceRT')
healthy_arrow = collect_means(dataframes_same, 'arrowRT')
epileptic_space = collect_means(dataframes_epileptic_same, 'spaceRT')
epileptic_arrow = collect_means(dataframes_epileptic_same, 'arrowRT')

def collect_means(df_list_same, df_list_diff, rt_col):
    vals_all = {dist: {"same": [], "diff": []} for dist in distributions_to_show}

    # SAME trials
    for df in df_list_same:
        df[rt_col] = pd.to_numeric(df[rt_col], errors='coerce')
        for dist in distributions_to_show:
            vals = remove_outliers(df[df['distribution'] == dist][rt_col])
            if not vals.empty:
                vals_all[dist]["same"].append(vals.mean())

    # DIFFERENT trials
    for df in df_list_diff:
        df[rt_col] = pd.to_numeric(df[rt_col], errors='coerce')
        for dist in distributions_to_show:
            vals = remove_outliers(df[df['distribution'] == dist][rt_col])
            if not vals.empty:
                vals_all[dist]["diff"].append(vals.mean())

    return vals_all
###################################################################################################################################

# --- Healthy
healthy_space = collect_means(dataframes_same, dataframes_diff, "spaceRT")
healthy_arrow = collect_means(dataframes_same, dataframes_diff, "arrowRT")

# --- Epileptic
epileptic_space = collect_means(dataframes_epileptic_same, dataframes_epileptic_diff, "spaceRT")
epileptic_arrow = collect_means(dataframes_epileptic_same, dataframes_epileptic_diff, "arrowRT")

# -------------------------------------------------
# Plotting
fig, axes = plt.subplots(2, 2, figsize=(6,6), dpi=300)


def plot_subplot(ax, vals_all, ylabel):
    positions = []
    labels = []
    p_values = []
    bar_width = 0.3
    spacing = 2.0
    pair_gap = 0.2  # <-- extra spacing between same/diff bars

    for i, dist in enumerate(distributions_to_show):
        pos_center = i * spacing
        pos_same = pos_center - bar_width/2 - pair_gap/2
        pos_diff = pos_center + bar_width/2 + pair_gap/2

        positions.extend([pos_same, pos_diff])
        labels.extend([f"same", f"diff"])

        # SAME box
        ax.boxplot(
            vals_all[dist]["same"],
            positions=[pos_same],
            widths=bar_width,
            patch_artist=True,
            boxprops=dict(facecolor='none', color='black', linewidth=0.5),
            medianprops=dict(color='black', linewidth=0.5),
            whiskerprops=dict(color='black', linewidth=0.5),
            capprops=dict(color='black', linewidth=0.5),
            showfliers=False
        )
        jitter_x = np.random.normal(pos_same, 0.05, size=len(vals_all[dist]["same"]))
        jitter_y = np.array(vals_all[dist]["same"]) + np.random.normal(0, 1, size=len(vals_all[dist]["same"]))
        ax.scatter(jitter_x, jitter_y, s=10, color=colors[i], alpha=0.5, edgecolors='none')

        # DIFFERENT box
        ax.boxplot(
            vals_all[dist]["diff"],
            positions=[pos_diff],
            widths=bar_width,
            patch_artist=True,
            boxprops=dict(facecolor='none', color='black', linewidth=0.5),
            medianprops=dict(color='black', linewidth=0.5),
            whiskerprops=dict(color='black', linewidth=0.5),
            capprops=dict(color='black', linewidth=0.5),
            showfliers=False
        )
        jitter_x = np.random.normal(pos_diff, 0.05, size=len(vals_all[dist]["diff"]))
        jitter_y = np.array(vals_all[dist]["diff"]) + np.random.normal(0, 1, size=len(vals_all[dist]["diff"]))
        ax.scatter(jitter_x, jitter_y, s=10, color=colors[i], alpha=0.5, edgecolors='none')

        # Permutation test SAME vs DIFFERENT
        same = vals_all[dist]["same"]
        diff = vals_all[dist]["diff"]
        if same and diff:
            combined = np.concatenate([same, diff])
            n_same = len(same)
            observed_diff = np.mean(same) - np.mean(diff)
            perm_diffs = [
                np.mean(np.random.permutation(combined)[:n_same]) -
                np.mean(np.random.permutation(combined)[n_same:])
                for _ in range(5000)
            ]
            p_val = np.mean(np.abs(perm_diffs) >= np.abs(observed_diff))
            p_values.append(p_val)
        else:
            p_values.append(1.0)

    # Correction
    reject, pvals_corrected, _, _ = multipletests(p_values, method='bonferroni')

    # Significance bars
    y_sig = max(ax.get_ylim()) * 0.95
    for i, dist in enumerate(distributions_to_show):
        pos_center = i * spacing
        pos_same = pos_center - bar_width/2 - pair_gap/2
        pos_diff = pos_center + bar_width/2 + pair_gap/2
        add_sig_bar(ax, pos_same, pos_diff, y_sig, pvals_corrected[i])

    ax.set_xticks(positions)
    ax.set_xticklabels(labels, rotation=45, ha='right')
    ax.set_ylabel(ylabel)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)


# -------------------------------------------------
# Fill subplots
plot_subplot(axes[0,0], healthy_space, "flip RT (ms)")
plot_subplot(axes[0,1], healthy_arrow, "choice RT (ms)")
plot_subplot(axes[1,0], epileptic_space, "flip RT (ms)")
plot_subplot(axes[1,1], epileptic_arrow, "choice RT (ms)")

axes[0,0].set_title("healthy - flip RT")
axes[0,1].set_title("healthy - choice RT")
axes[1,0].set_title("epileptic - flip RT")
axes[1,1].set_title("epileptic - choice RT")


# plt.tight_layout()
# plt.savefig(os.path.join(output_folder, "same_different_trials.pdf"),
#             format="pdf", dpi=300, bbox_inches="tight")
# plt.savefig(os.path.join(output_folder, "same_different_trials.svg"),
#             format="svg", dpi=300, bbox_inches="tight")
# plt.show()


Text(0.5, 1.0, 'epileptic - choice RT')

# DEBUG